# Week 5 final report

Week 5 Final Report
Opening a New Shopping Mall in Kuala Lumpur, Malaysia

* Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web * * * scraping the data from Wikipedia page
* Get the geographical coordinates of the neighborhoods
* Obtain the venue data for the neighborhoods from Foursquare API
* Explore and cluster the neighborhoods
* Select the best cluster to open a new shopping mall


## 1. Import Libraries

In [3]:
!pip install geocoder
!pip install folium

     |████████████████████████████████| 102kB 6.8MB/s ta 0:00:011
     |████████████████████████████████| 102kB 10.1MB/s ta 0:00:01


In [4]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium
print("Libraries imported")

Libraries imported


## 2. scrap data from wikipedia page into dataframe

In [5]:
data=requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [6]:
soup=BeautifulSoup(data,'html.parser')

In [7]:
neighborhoodlist=[]

In [8]:
for row in soup.find_all("div",class_="mw-category")[0].findAll("li"):
    neighborhoodlist.append(row.text)

In [9]:
kl_df=pd.DataFrame({'Neighborhood':neighborhoodlist})
kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [10]:
kl_df.shape

(71, 1)

## get geographical cordinates

In [11]:
#define a function to get cordinate
def get_latlng(neighborhood):
    #initialize your variable with None
    lat_lng_coords=None
    #loop until you get the cordinates
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{}, Kaula Lumpur,Malaysia'.format(neighborhood))
        lat_lng_coords=g.latlng
    return lat_lng_coords
    

In [12]:
coords=[get_latlng(neighborhood) for neighborhood in kl_df['Neighborhood'].tolist()]

In [13]:
coords

[[3.0576900000000364, 101.74388000000005],
 [3.1484988508598852, 101.69672774991264],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1292000000000257, 101.67844000000008],
 [3.111020000000053, 101.66283000000004],
 [3.061870000000056, 101.74675000000008],
 [3.147890000000075, 101.69405000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.0578100000000745, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.1486092228953293, 101.69985445868105],
 [3.129290000000026, 101.69896000000006],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.1479700000000435, 101.66795000000008],
 [3.1364442625375997, 101.69029430612223],
 [3.138766317393615, 101.68401504688993],
 [3.1578250453611005, 101.

In [14]:
df_coords=pd.DataFrame(coords,columns=['Latitude','Longitude'])

In [15]:
kl_df['Latitude']=df_coords['Latitude']
kl_df['Longitude']=df_coords['Longitude']

In [16]:
print(kl_df.shape)
kl_df.head()

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [17]:
kl_df.to_csv('kl_df.csv',index=False)

## 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [18]:
#get the coordinate of kuala lampur
address='Kuala Lumpur,Malaysia'

geolocator=Nominatim(user_agent="my-application")
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geograpical coordinate of Kaula Lumpur, Malaysia {},{}.'.format(latitude,longitude))

The geograpical coordinate of Kaula Lumpur, Malaysia 3.1516964,101.6942371.


In [19]:
#create map of kaula lampur using lat and long value
map_kl=folium.Map(location=[latitude,longitude],zoom_start=11)

#add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'],kl_df['Longitude'],kl_df['Neighborhood']):
    label='{}'.format(neighborhood)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7).add_to(map_kl)
    
map_kl

In [20]:
#save the map as Html file
map_kl.save('map_kl.html')

## 5. Use the Foursquare API to explore the neighborhoods

In [21]:
#define Foursquare Credential and Version
#define Foursquare Credentials and Version
CLIENT_ID='BMO3Z2X2BA3P4W00553PUMKFNNXCQCBYNWKNJV5QPKTROM5B' #your Foursquare Id
CLIENT_SECRET='OG4BUOB4XLYNBCA1YNNWBYF54YX1UT0ZIEGJP3E1JVHC3CUG' #your Foursquare Secret
VERSION='20200708' # Foursquare API version Date

print('Your credential:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+ CLIENT_SECRET)

Your credential:
CLIENT_ID: BMO3Z2X2BA3P4W00553PUMKFNNXCQCBYNWKNJV5QPKTROM5B
CLIENT_SECRET: OG4BUOB4XLYNBCA1YNNWBYF54YX1UT0ZIEGJP3E1JVHC3CUG


In [22]:
#now let get top 100 venues that are within a radius of 2000 meters

radius=2000
LIMIT=100

venues=[]

for lat,long,neighborhood in zip(kl_df['Latitude'],kl_df['Longitude'],kl_df['Neighborhood']):
    
#create the API request URL
    url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}".format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    lat,
    long,
    radius,
    LIMIT)
    
#make the GET request
    results=requests.get(url).json()["response"]["groups"][0]['items']
    
# return only relevant information for each nearby venue

    for venue in results:
        venues.append((
        neighborhood,
        lat,
        long,
        venue['venue']['name'],
        venue['venue']['location']['lat'],
        venue['venue']['location']['lng'],
        venue['venue']['categories'][0]['name']))

In [23]:
#convert the venues list into a new DataFrame
venues_df=pd.DataFrame(venues)

#define the column names
venues_df.columns=['Neighborhood','Latitude','Longitude','VenueName','VenueLatitude','VenueLongitude','VenueCategory']

print(venues_df.shape)
venues_df.head()

(2130, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
2,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,Ivy Sekinchan Seafood Noodle House 适耕莊特制魚丸海鲜面,3.065749,101.748718,Noodle House


In [24]:
#let's check how many venues were returned for each neighborhood

venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,30,30,30,30,30,30
"Ampang, Kuala Lumpur",30,30,30,30,30,30
Bandar Menjalara,30,30,30,30,30,30
Bandar Sri Permaisuri,30,30,30,30,30,30
Bandar Tasik Selatan,30,30,30,30,30,30
Bandar Tun Razak,30,30,30,30,30,30
Bangsar,30,30,30,30,30,30
Bangsar Park,30,30,30,30,30,30
Bangsar South,30,30,30,30,30,30


In [25]:
#Lets find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 227 uniques categories.


In [26]:
#print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Food & Drink Shop', 'Noodle House',
       'Seafood Restaurant', 'Chinese Restaurant', 'Breakfast Spot',
       'Restaurant', 'Food Court', 'Vegetarian / Vegan Restaurant',
       'Bubble Tea Shop', 'Pet Store', 'Park', 'Other Great Outdoors',
       'Asian Restaurant', 'Indian Restaurant', 'Coffee Shop', 'Spa',
       'Convenience Store', 'Food Truck', 'Dim Sum Restaurant',
       'Japanese Restaurant', 'Snack Place', 'Dessert Shop',
       'Outlet Store', 'Hostel', 'South Indian Restaurant', 'Exhibit',
       'Chettinad Restaurant', 'Hotel', 'Monument / Landmark',
       'Juice Bar', 'Speakeasy', 'Athletics & Sports',
       'Latin American Restaurant', "Men's Store", 'Flower Shop',
       'Kebab Restaurant', 'Museum', 'Gym', 'Fried Chicken Joint',
       'IT Services', 'Plaza', 'Mosque', 'Fast Food Restaurant',
       'Ice Cream Shop', 'Spanish Restaurant', 'Café', 'Food Stand',
       'Bistro', 'Lounge'], dtype=object)

In [27]:
#check if the result contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

## 6. Analyze Each Neighborhood

In [28]:
#one hot encoding
kl_onehot=pd.get_dummies(venues_df[['VenueCategory']],prefix="",prefix_sep="")

#add neighborhood column back to dataframe
kl_onehot['Neighborhoods']=venues_df['Neighborhood']

#move neighborhood column to first colmn
fixed_columns=[kl_onehot.columns[-1]]+list(kl_onehot.columns[:-1])
kl_onehot=kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2130, 228)


,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Cantonese Restaurant,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Donut Shop,Dumpling Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [29]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

kl_grouped=kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(kl_grouped.shape)
kl_grouped

(71, 228)


,Neighborhoods,Accessories Store,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Beer Bar,Beer Garden,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Café,Camera Store,Cantonese Restaurant,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Donut Shop,Dumpling Restaurant,Electronics Store,Exhibit,Fabric Shop,Farmers Market,Fast Food Restaurant,Field,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health & Beauty Service,Health Food Store,High School,Hill,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Neighborhood,Night Market,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Racetrack,Residential Building (Apartment / Condo),Resort,Restaurant,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Alam Damai,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [30]:
len(kl_grouped[kl_grouped["Shopping Mall"]>0])

22

In [31]:
#Create a new DataFrame For Shopping Mall Data only

kl_mall=kl_grouped[["Neighborhoods","Shopping Mall"]]

In [32]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.0
1,"Ampang, Kuala Lumpur",0.0
2,Bandar Menjalara,0.0
3,Bandar Sri Permaisuri,0.0
4,Bandar Tasik Selatan,0.0


## 7. Cluster Neighborhoods

Run K-means to cluster the neighborhoods in Kaula Lumpur into 3 Cluster

In [33]:
# set number of cluster
kclusters=3
kl_clustering=kl_mall.drop(["Neighborhoods"],1)

#run k-mean clustering
kmeans=KMeans(n_clusters=kclusters,random_state=0).fit(kl_clustering)

#check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [34]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

kl_merged=kl_mall.copy()

# add clustering labels

kl_merged["Cluster Labels"]=kmeans.labels_

In [35]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"},inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.0,1
1,"Ampang, Kuala Lumpur",0.0,1
2,Bandar Menjalara,0.0,1
3,Bandar Sri Permaisuri,0.0,1
4,Bandar Tasik Selatan,0.0,1


In [36]:
#merge neighborhood with shopping mall ta add latitude and longitude for each neighborhood

kl_merged=kl_merged.join(kl_df.set_index("Neighborhood"),on="Neighborhood")

print(kl_merged.shape)

#check last column
kl_merged.head()

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,1,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.0,1,3.148499,101.696728
2,Bandar Menjalara,0.0,1,3.190350,101.625450
3,Bandar Sri Permaisuri,0.0,1,3.103910,101.712260
4,Bandar Tasik Selatan,0.0,1,3.072750,101.714610


In [37]:
#sort the result by cluster labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"],inplace=True)
kl_merged

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
8,Bangsar South,0.066667,0,3.111020,101.662830
66,Taman Tun Dr Ismail,0.066667,0,3.152830,101.622710
46,Segambut,0.066667,0,3.186390,101.668100
30,KL Eco City,0.066667,0,3.117130,101.673840
12,Bukit Bintang,0.066667,0,3.147770,101.708550
27,"Jalan Cochrane, Kuala Lumpur",0.066667,0,3.134630,101.721690
44,Putrajaya,0.066667,0,3.125851,101.718513
47,Semarak,0.000000,1,3.179943,101.721449
45,Salak South,0.000000,1,3.081540,101.696890
43,"Pudu, Kuala Lumpur",0.000000,1,3.133540,101.713070


In [ ]:
# visualize the resulting cluster
#create map

map_clusters=folium.Map(location=[latitude,longitude],zoom_start=11)

#set color scheme for clusters
x=np.arange(kcl)